In [ ]:
import mlutils
import warnings
import os
import pandas as pd
warnings.filterwarnings(action="ignore")
os.getcwd()

station = pd.read_csv(
    "assets/seoul_bike_station_01_12.csv", encoding="CP949", index_col=0
)
near_bus = pd.read_csv(
    "assets/near_bus_500m.csv", encoding="CP949", index_col=0
)
seoul_bike = pd.read_parquet(
    "/Users/yangwoolee/git_repo/seoul_bike_dash/parquet/220607_bike_record.parquet.gzip"
)
sub_info = pd.read_csv(
    "assets/sub_and_bike_info.csv", encoding="CP949", index_col=0
)

search_info = pd.read_csv(
    "assets/search_info.csv", encoding="CP949", index_col=0
)
moon_light = mlutils.moon_light(station,near_bus,seoul_bike,sub_info)

In [3]:
import ast
import numpy as np

def near_bus_bike_info( bus_route: pd.DataFrame, arrival_info: pd.DataFrame
    ) -> pd.DataFrame:
        """
        추천 대여소 정보를 만드는 함수임
        bus_route : 버스 도착지 정보
        arrival_info : 대여소 도착지 정보
        """
        bike_id = bus_route.iloc[-1]["bike_id"]  # bike_id는 무조건 리스트 형태로
        bike_station_id = arrival_info.index[0]

        # # 추천_대여소_선정 : ast : '[123,]' => [123]으로 만들어줌 --> list
        # if type(bike_id) == str:
        #     bike_id = ast.literal_eval(bike_id)

        bike_id = ast.literal_eval(bike_id)

        raw = moon_light.raw_data(bike_station_id)

        br_record_id = raw.groupby("st_id1").size().index

        ### 50건 이상만 list에 담기도록 설정
        # br_record_id = br_record[br_record > 10].index


        recom_bike_id = np.intersect1d(bike_id, br_record_id).tolist()

        if pd.Series(recom_bike_id).empty:
            return pd.Series([]) # empty

        # 버스정류장 근처 대여소와 집 근처 대여소 거리 비교
        # ending point를 외부에서 받아야하네 (클래스로 함수를 만들어야하나.)

        # # 추천 대여소 리스트 만들기
        BM_2 = (moon_light.seoul_bike["st_id2"] == bike_station_id) & (
            moon_light.seoul_bike["st_id1"].isin(recom_bike_id)
        )
        BM_1 = (moon_light.seoul_bike["st_id1"] == bike_station_id) & (
            moon_light.seoul_bike["st_id2"].isin(recom_bike_id)
        )
        seoul_bike_sorting = moon_light.seoul_bike.loc[BM_1 | BM_2]  # 거리 및 시간 계산을 위해 따로 추출


        bike_recommend = []
        for starting_station in recom_bike_id:
            BM_1 = (seoul_bike_sorting["st_id1"] == starting_station) & (
                seoul_bike_sorting["st_id2"] == bike_station_id
            )
            BM_2 = (seoul_bike_sorting["st_id2"] == starting_station) & (
                seoul_bike_sorting["st_id1"] == bike_station_id
            )

            # 평균시간 계산(column : time)
            mean_station = (
                seoul_bike_sorting[BM_1 | BM_2]["riding_time"]
                .value_counts()
                .sort_values(ascending=False)[:5]
                .index.values.mean()
            )

            # 이용기록 계산(column : num)
            num_station = len(seoul_bike_sorting[BM_1 | BM_2])

            # 직선거리 계산(column : dist)
            lat_1 = moon_light.station.query("st_id==@starting_station")["latitude"].values
            lon_1 = moon_light.station.query("st_id==@starting_station")["longtitude"].values
            lat_2 = moon_light.station.query("st_id==@bike_station_id")["latitude"].values
            lon_2 = moon_light.station.query("st_id==@bike_station_id")["longtitude"].values
            dist_station = moon_light.haversine_np(lat_1, lon_1, lat_2, lon_2)

            # 자료 저장
            station_info = moon_light.station.query("st_id==@starting_station")
            bike_recommend.append(
                [
                    station_info["st_name"].iloc[0],
                    station_info["latitude"].iloc[0],
                    station_info["longtitude"].iloc[0],
                    starting_station,
                    mean_station,
                    num_station,
                    f"{dist_station[0]/1000 :.2f}km",
                ]
            )

        # return
        bike_recommend = pd.DataFrame(
            bike_recommend,
            columns=[
                "st_name",
                "latitude",
                "longtitude",
                "st_id",
                "time",
                "num",
                "dist",
            ],
        )
        bike_recommend = bike_recommend[bike_recommend['num']>50]

        return bike_recommend # 또는 empty

dep_info = moon_light.departure_info(5290)  # dataframe
arr_info = moon_light.arrival_info(1968)
abb = near_bus_bike_info(dep_info,arr_info)
abb

Series([], dtype: float64)

In [26]:
a =[]
pd.Series(a).empty

True

In [4]:
bike= search_info[search_info['value'] <3000]

for i in range(100) :
    dep = search_info.sample(1)
    dep_id = dep['value'].iloc[0]
    arr = bike.sample(1)
    arr_id = arr.index[0] 
    try : 
        moon_light.route_data(dep_id,arr_id)
    except :
        print('error :',dep, arr)
        break
print('문제 없음!!')


버스
목적지와 연결된 검색노선 없음.
해당 목적지로 이동가능한 야간버스가 없습니다.
버스
목적지와 연결된 검색노선 없음.
해당 목적지로 이동가능한 야간버스가 없습니다.
자전거
지하철
버스
목적지와 연결된 검색노선 없음.
해당 목적지로 이동가능한 야간버스가 없습니다.
자전거
자전거
자전거
버스
목적지와 연결된 검색노선 없음.
해당 목적지로 이동가능한 야간버스가 없습니다.
자전거
자전거
버스
목적지와 연결된 검색노선 없음.
해당 목적지로 이동가능한 야간버스가 없습니다.
자전거
자전거
버스
목적지와 연결된 검색노선 없음.
해당 목적지로 이동가능한 야간버스가 없습니다.
자전거
자전거
자전거
자전거
자전거
자전거
자전거
버스
목적지와 연결된 검색노선 없음.
해당 목적지로 이동가능한 야간버스가 없습니다.
자전거
버스
목적지와 연결된 검색노선 없음.
해당 목적지로 이동가능한 야간버스가 없습니다.
자전거
버스
목적지와 연결된 검색노선 없음.
해당 목적지로 이동가능한 야간버스가 없습니다.
지하철
버스
ending_order                                       bike_id  bus  order     id  \
1280       [742,774,1115,1132,1135,1170,5075]  N62     22  15128   
1393  [742,774,1115,1132,1135,1136,1170,5075]  N62    140  16173   

              name  longtitude   latitude        dist  
1280   목3동시장.강서도서관  126.863937  37.549695  829.465536  
1393  영일고등학교.강서도서관  126.863355  37.548989  905.162885  
dep_arr_bus_info
                                      bike_id  bus order     id          name  \
1381          

In [4]:

# id를 대여소 번호로 바꿔줌
def arrival_info( arr_id: int) -> pd.DataFrame:
    return station.iloc[[arr_id]]

a = mlutils.search_by_text(search_info,'N64 현대아파트')
# moon_light.departure_info(4508)

arrival_info(1389)

,st_name,district,latitude,longtitude
st_id,,,,
2020,장승배기역2번출구뒷편쪽,동작구,37.504498,126.938988


> ### 디버그 용도

In [7]:
import mlutils
import warnings
import os
import pandas as pd

warnings.filterwarnings(action="ignore")
os.getcwd()

station = pd.read_csv(
    "assets/seoul_bike_station_01_12.csv", encoding="CP949", index_col=0
)
near_bus = pd.read_csv("assets/near_bus_500m.csv", encoding="CP949", index_col=0)
seoul_bike = pd.read_parquet(
    "/Users/yangwoolee/git_repo/seoul_bike_dash/parquet/220607_bike_record.parquet.gzip"
)
sub_info = pd.read_csv("assets/sub_and_bike_info.csv", encoding="CP949", index_col=0)

search_info = pd.read_csv("assets/search_info.csv", encoding="CP949", index_col=0)
moon_light = mlutils.moon_light(station, near_bus, seoul_bike, sub_info)

#  def route_data(dep_id, arr_id):

dep_id = 5541
arr_id = 1389
dep_info = moon_light.departure_info(dep_id)  # dataframe
arr_info = moon_light.arrival_info(arr_id)  # dataframe


if dep_id < 3000:
    print("자전거")
    bike = moon_light.route_coor(dep_info, arr_info)
    record_info = moon_light.record_info(dep_info.index[0], arr_info.index[0], 50)
    # [{"bike": ["당산 육갑문", "목동1단지아파트 118동 앞"]}]
    route_info = [
        {
            "bike": [
                dep_info["st_name"].iloc[0],
                arr_info["st_name"].iloc[0],
                record_info[0],
                record_info[1],
            ]
        }
    ]

    retu = dict(bike=bike, route_info=route_info)

elif dep_id > 4000:
    print("버스")

    # * -- 출도착 버스 정보-- *
    dep_arr_bus_info = moon_light.finding_start_end(dep_info, arr_info)
    print(dep_arr_bus_info)


    # * -- 경로 선택 및 경로 노선 추출 -- *
    bus_whole, waypoint, bus_start_end = moon_light.bus_route_info(dep_arr_bus_info)
    if bus_start_end == [] :
        bus=bus,
        walk=walk,
        bike=bike,
        
    else
    # * -- 좌표 받기 -- *
    arr_trans = bus_start_end.iloc[[1]]

    arr_bike_info = moon_light.near_bus_bike_info(arr_trans, arr_info).sort_values(
        by="num", ascending=False
    )

    bus = moon_light.bus_route_coor(bus_start_end, waypoint)
    walk = moon_light.route_coor(arr_trans, arr_bike_info.iloc[[0]])
    bike = moon_light.route_coor(arr_bike_info.iloc[[0]], arr_info)
    record_info = moon_light.record_info(
        arr_bike_info["st_id"].iloc[0], arr_info.index[0], 50
    )

    # # -- 자전거 길 네이버 지도로도 표현 -- #
    # a = arr_bike_info.iloc[[0]][["latitude", "longtitude"]]
    # b = arr_info.reset_index()[["latitude", "longtitude"]]
    # start_end = pd.concat([a, b], axis=0)
    # bus # bus_direct
    route_info = [
        {
            "bus": [
                bus_start_end["name"].iloc[0],
                bus_start_end["name"].iloc[1],
            ],
            "bike": [
                arr_bike_info["st_name"].iloc[0],
                arr_info["st_name"].iloc[0],
                record_info[0],
                record_info[1],
            ],
        },
    ]
    dep_bike_info = moon_light.near_bus_bike_info(
        bus_start_end.iloc[[0]], arr_info
    ).sort_values(by="num", ascending=False)

    if dep_bike_info.empty == False:
        bike2 = moon_light.route_coor(dep_bike_info, arr_info)
        record_info2 = moon_light.record_info(
            dep_bike_info["st_id"].iloc[0], arr_info.index[0], 50
        )
        bike2 = (
            {
                "bike2": [
                    dep_bike_info["st_name"].iloc[0],
                    arr_info["st_name"].iloc[0],
                    record_info2[0],
                    record_info2[1],
                ]
            },
        )
        route_info.extend(bike2)

    retu = dict(
        bus=bus,
        walk=walk,
        bike=bike,
        bike2=bike2,
        route_info=route_info,
    )

else:
    print("지하철")
    # * -- 자전거 대여소 정보 추출 -- *
    near_bus_bike_info = moon_light.near_bus_bike_info(dep_info, arr_info).sort_values(
        by="dist"
    )
    if near_bus_bike_info.empty :
        print('near_bus_bike_info is empty')
    departure_bike_station = pd.DataFrame(near_bus_bike_info.iloc[0]).T

    # * -- 자전거 경로(사실 보행로 추천이라는 사실~)-- *
    sub = moon_light.route_coor(dep_info, departure_bike_station)
    bike = moon_light.route_coor(departure_bike_station, arr_info)
    record_info = moon_light.record_info(
        departure_bike_station["st_id"].iloc[0], arr_info.index[0], 50
    )
    # sub
    # [{"sub": ["당산역"], "bike": ["당산 육갑문", "목동1단지아파트 118동 앞"]}]
    route_info = [
        {
            "sub": [dep_info["sub_name"].iloc[0]],
            "bike": [
                departure_bike_station["st_name"].iloc[0],
                arr_info["st_name"].iloc[0],
                record_info[0],
                record_info[1],
            ],
        }
    ]
    retu = dict(sub=sub, bike=bike, route_info=route_info)


버스
                    bike_id  bus  order     id        name  longtitude  \
1541  [2007,2065,4175,2081]  N64    126  20008  현대아파트.유한양행  126.932374   

       latitude  
1541  37.513237  
버스를 타지 않았네


AttributeError: 'list' object has no attribute 'iloc'

In [10]:
dep_arr_bus_info
print(dep_info)
print(arr_info)

                    bike_id  bus  order     id        name  longtitude  \
1541  [2007,2065,4175,2081]  N64    126  20008  현대아파트.유한양행  126.932374   

       latitude  
1541  37.513237  
            st_name district   latitude  longtitude
st_id                                              
2020   장승배기역2번출구뒷편쪽      동작구  37.504498  126.938988


In [ ]:

    # * -- 경로 선택 및 경로 노선 추출 -- *
    bus_whole, waypoint, bus_start_end = moon_light.bus_route_info(dep_arr_bus_info)

    # * -- 좌표 받기 -- *


    arr_trans = bus_start_end.iloc[[1]]

    dep_bike_info = moon_light.near_bus_bike_info(
    bus_start_end.iloc[[0]], arr_info
    ).sort_values(by="num", ascending=False)

    arr_bike_info = moon_light.near_bus_bike_info(
    arr_trans, arr_info
    ).sort_values(by="num", ascending=False)

    dep_arr_bike_coor = moon_light.route_coor(dep_bike_info, arr_info)


    bus_coor = moon_light.bus_route_coor(bus_start_end, waypoint)
    walk_coor = moon_light.route_coor(
        arr_trans, arr_bike_info.iloc[[0]]
    )
    bike_coor = moon_light.route_coor(arr_bike_info.iloc[[0]], arr_info)
    retu= dict(
        dep_arr_bike_coor=dep_arr_bike_coor,
        bus_coor=bus_coor,
        walk_coor=walk_coor,
        bike_coor=bike_coor,
    )
else:
    print("지하철")
    # * -- 자전거 대여소 정보 추출 -- *
    near_bus_bike_info = moon_light.near_bus_bike_info(
        dep_info, arr_info
    ).sort_values(by="dist")
    departure_bike_station = pd.DataFrame(near_bus_bike_info.iloc[0]).T

    # * -- 자전거 경로(사실 보행로 추천이라는 사실~)-- *
    walk_coor = moon_light.route_coor(dep_info, departure_bike_station)
    bike_coor = moon_light.route_coor(departure_bike_station, arr_info)
    retu= dict(
        walk_coor=walk_coor, bike_coor=bike_coor
    )

> ### Card에 넣을 내용

In [4]:

from typing import Dict

def record_check(dep_id: int,arr_id: int,count : int = 50) -> pd.DataFrame :
    BM = (seoul_bike['st_id1'] == dep_id) & (seoul_bike['st_id2'] == arr_id)
    raw_data_1 = seoul_bike[BM]
    BM = (seoul_bike['st_id1'] == arr_id) & (seoul_bike['st_id2'] == dep_id)
    raw_data_2 = seoul_bike[BM]
    concat_data = pd.concat([raw_data_1,raw_data_2],axis=0)

    if len(concat_data) <= count :
        return pd.DataFrame([])

    
    all_time = concat_data['riding_time'].value_counts().sort_values(ascending=False) # 도달한 시간대
    total_borrow =len(concat_data)

    k = []
    i = 2
    # 일정 비율 이상인 시간대만 구하기
    while len(k) < 1:
        k = all_time[all_time >= (total_borrow / i)]
        i = i * 1.5
        
    ### 대여시간대
    time_line = k.index

    ### 대여기록
    counts = k.values

    total_counts = k.sum()

    ### 대여시간 * 대여기록
    total_time = sum([a * b for a, b in zip(time_line, counts)])

    # 평균 시간
    avg_time = total_time / total_counts

    # 올림
    result = round(avg_time, 0)

    return [result,total_borrow]
dep_info = moon_light.departure_info(4443)
arr_info = moon_light.arrival_info(528)


record_check(arr_bike_info['st_id'].iloc[0],arr_info.index[0])

NameError: name 'arr_bike_info' is not defined

In [28]:
# bus # bus_direct
[{"bus":[bus_start_end['name'].iloc[0],bus_start_end['name'].iloc[1]],'bike':[arr_bike_info['st_name'].iloc[0], arr_info['st_name'].iloc[0]]}, {'bike2' : [dep_bike_info['st_name'].iloc[0], arr_info['st_name'].iloc[0]]}]

# sub
[{"sub":['당산역'],'bike':['당산 육갑문', '목동1단지아파트 118동 앞']}]
[{"sub":[dep_info['sub_name'].iloc[0]],'bike':[departure_bike_station['st_name'].iloc[0], arr_info['st_name'].iloc[0]]}]


# bike
[{'bike':['당산 육갑문', '목동1단지아파트 118동 앞']}]
[{'bike' : [dep_info['name'].iloc[0],arr_info['st_name'].iloc[0]]}]

33.0

In [8]:
print("bus_start_end", bus_start_end['name'].iloc[0])
print('')
print("arr_trans", arr_trans.values)
print('')
print("arr_bike_info", arr_bike_info['st_name'])
print('')
print("arr_info", arr_info['st_name'].iloc[0])
print('')
print("dep_info", dep_info)
print('')
print("dep_bike_info", dep_bike_info)
print('')
print("departure_bike_station", departure_bike_station)
print('')

bus_start_end 당산역

arr_trans [['염창역.서울도시가스' 126.875881 37.547097 112 '[700,773,1169,1171,2703,3761]']]

arr_bike_info 1    염창역 3, 4번 출구 사이
2          염창역 1번 출구
0    KB국민은행 염창역 지점 앞
Name: st_name, dtype: object

arr_info 목동1단지아파트 118동 앞 (월촌초등학교 정류소 옆

dep_info                         bike_id  bus  order     id name  longtitude   latitude
443  [247,267,268,272,290,3201]  N26     24  19172  당산역  126.900719  37.534589

dep_bike_info   st_name   latitude  longtitude  st_id  time  num    dist
0   당산육갑문  37.535339  126.903679    272  33.0  109  3.03km



NameError: name 'departure_bike_station' is not defined

> ### 이름으로 버스 정류장과 대여소 선택